In [3]:
import threading

class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

class LinkedList:
    def __init__(self, max_size=None):
        self.head = None
        self.size = 0
        self.max_size = max_size  # Maximum size limit for the linked list
        self.lock = threading.Lock()  # Lock for concurrency control

    def append(self, data):
        # Validate input data
        if len(data) > 1000:  # Example: Limit data size to prevent excessively large payloads
            raise ValueError("Data size exceeds maximum limit")

        with self.lock:
            # Rate limiting mechanism can be implemented here
            if self.max_size is not None and self.size >= self.max_size:
                raise ValueError("Linked list is full")

            new_node = Node(data)
            if self.head is None:
                self.head = new_node
            else:
                last = self.head
                while last.next:
                    last = last.next
                last.next = new_node
            self.size += 1

    def print_list(self):
        current = self.head
        while current:
            print(current.data, end=" ")
            current = current.next


In [2]:
# Example usage
list = LinkedList(max_size=1000)  # Set maximum size limit
list.append("Data 1")
list.append("Data 2")
list.print_list()

Data 1 Data 2 

In [4]:
import threading

class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

class LinkedList:
    def __init__(self, max_size=None):
        self.head = None
        self.size = 0
        self.max_size = max_size
        self.lock = threading.Lock()

    def append(self, data):
        if len(data) > 1000:
            raise ValueError("Data size exceeds maximum limit")

        with self.lock:
            if self.max_size is not None and self.size >= self.max_size:
                raise ValueError("Linked list is full")

            new_node = Node(data)
            if not self.head:
                self.head = new_node
            else:
                last = self.head
                while last.next:
                    last = last.next
                last.next = new_node
            self.size += 1

    def remove(self, data):
        """
        删除链表中第一个值等于 data 的节点。
        如果找到节点则删除并返回 True，否则返回 False。
        """
        with self.lock:
            prev = None
            curr = self.head

            # 遍历寻找目标节点
            while curr:
                if curr.data == data:
                    # 如果是头节点
                    if prev is None:
                        self.head = curr.next
                    else:
                        prev.next = curr.next
                    self.size -= 1
                    return True
                prev = curr
                curr = curr.next

            # 未找到
            return False

    def print_list(self):
        current = self.head
        while current:
            print(current.data, end=" ")
            current = current.next
        print()

In [6]:
# 示例
if __name__ == "__main__":
    lst = LinkedList(max_size=5)
    lst.append("A")
    lst.append("B")
    lst.append("C")
    print("删除前：", end="")
    lst.print_list()   # 输出: A B C 

    removed = lst.remove("B")
    print("尝试删除 'B'：", "成功" if removed else "失败")
    print("删除后：", end="")
    lst.print_list()   # 输出: A C 

    # 删除头节点
    lst.remove("A")
    print("再删除 'A' 后：", end="")
    lst.print_list()   # 输出: C 


删除前：A B C 
尝试删除 'B'： 成功
删除后：A C 
再删除 'A' 后：C 


In [7]:
import threading
import time

class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

class TokenBucket:
    """
    简单令牌桶：rate=令牌生成速率（tokens/sec），capacity=桶大小。
    """
    def __init__(self, rate, capacity):
        self.rate = rate
        self.capacity = capacity
        self.tokens = capacity
        self.timestamp = time.monotonic()
        self.lock = threading.Lock()

    def consume(self, tokens=1):
        with self.lock:
            now = time.monotonic()
            # 根据时间流逝补充令牌
            delta = now - self.timestamp
            self.tokens = min(self.capacity, self.tokens + delta * self.rate)
            self.timestamp = now

            if self.tokens >= tokens:
                self.tokens -= tokens
                return True
            return False

class SafeLinkedList:
    def __init__(self, max_size=None, rate_limit=None):
        self.head = None
        self.tail = None            # 尾指针，用于 O(1) append
        self.size = 0
        self.max_size = max_size
        self.lock = threading.RLock()

        # 如果指定了 rate_limit（requests/sec），就启用令牌桶
        self.bucket = TokenBucket(rate_limit, rate_limit) if rate_limit else None

    def append(self, data):
        # 严格校验
        if not isinstance(data, str):
            raise ValueError("只支持字符串类型数据")
        if len(data) > 1000:
            raise ValueError("数据长度超过1000字符")

        # 速率限制
        if self.bucket and not self.bucket.consume():
            raise RuntimeError("操作过于频繁，请稍后重试")

        # 大小限制
        if self.max_size is not None and self.size >= self.max_size:
            raise RuntimeError("链表已达最大容量")

        # 尝试获取锁，最久等100ms
        acquired = self.lock.acquire(timeout=0.1)
        if not acquired:
            raise RuntimeError("系统繁忙，请稍后重试")
        try:
            node = Node(data)
            if self.head is None:
                # 空表时，head 和 tail 都指向新节点
                self.head = node
                self.tail = node
            else:
                # O(1) 插入到尾部
                self.tail.next = node
                self.tail = node
            self.size += 1
        finally:
            self.lock.release()

    def remove(self, data):
        # 速率限制
        if self.bucket and not self.bucket.consume():
            raise RuntimeError("操作过于频繁，请稍后重试")

        # 尝试获取锁
        acquired = self.lock.acquire(timeout=0.1)
        if not acquired:
            raise RuntimeError("系统繁忙，请稍后重试")
        try:
            prev, curr = None, self.head
            while curr:
                if curr.data == data:
                    # 找到就删除
                    if prev:
                        prev.next = curr.next
                    else:
                        self.head = curr.next

                    # 如果删的是尾节点，也要更新 tail
                    if curr is self.tail:
                        self.tail = prev

                    self.size -= 1
                    return True
                prev, curr = curr, curr.next
            return False
        finally:
            self.lock.release()

    def print_list(self):
        # 只读操作，可以不加锁或用 try-acquire 快速读
        curr = self.head
        while curr:
            print(curr.data, end=" ")
            curr = curr.next
        print()


In [8]:
# 示例
if __name__ == "__main__":
    lst = LinkedList(max_size=5)
    lst.append("A")
    lst.append("B")
    lst.append("C")
    print("删除前：", end="")
    lst.print_list()   # 输出: A B C 

    removed = lst.remove("B")
    print("尝试删除 'B'：", "成功" if removed else "失败")
    print("删除后：", end="")
    lst.print_list()   # 输出: A C 

    # 删除头节点
    lst.remove("A")
    print("再删除 'A' 后：", end="")
    lst.print_list()   # 输出: C 


删除前：A B C 
尝试删除 'B'： 成功
删除后：A C 
再删除 'A' 后：C 
